In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_csv('C:/data/CS/SA/threshold4_balanced_training_set.csv', usecols=['CONTENT','sentiment'])
df.head(10)
df['label']=df['sentiment']
df['label'].value_counts()

positive    1269
negative    1269
neutral     1269
Name: label, dtype: int64

In [4]:
for i, item in enumerate (df['label']):
  if item=='positive':
    df['label'][i]=2
  if item=='neutral':
    df['label'][i]=1
  if item=='negative':
    df['label'][i]=0
df['label'].value_counts()

2    1269
1    1269
0    1269
Name: label, dtype: int64

In [5]:
tdf = pd.read_csv('C:/data/CS/SA/test.csv')
len(tdf)
x = df['CONTENT']
y = df['label']
x_test=tdf['CONTENT']
y_test=tdf['label']


In [6]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2, random_state=SEED)
print (x_train)

3390    the stock market get hit with a double whammy ...
1297    this aerial photo take on january 2 2017 show ...
3320    doylestown pa jan 17 2019 globe newswire proph...
3466    new york business wire long term shareholder h...
1987    -pron- seem that the mass and most of the fina...
1711    investor in goodyear tire rubber co symbol gt ...
785     dublin oct 2 2019 prnewswire the passenger sec...
642     chicago july 8 2019 prnewswire the chicago qua...
747     this news release constitute a designate news ...
1792    oil price rise for four consecutive trading da...
855     herndon va oct 07 2019 globe newswire skybitz ...
309     new york oct 28 2019 globe newswire vornado re...
684     tempe ariz april 24 2019 prnewswire benchmark ...
1429    the late 13f reporting period have come and go...
338     for immediate release chicago il october 17 20...
758     san diego sept 12 2019 prnewswire l attitude a...
2644    howard marks put -pron- nicely when -pron- say...
710     mr kle

In [7]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [9]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [10]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████████████████████████████████████████████████████████████████████| 4177/4177 [00:00<00:00, 835420.71it/s]


In [11]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|█████████████████████████████████████████████████████████████████████████| 4177/4177 [00:00<00:00, 2088402.41it/s]


Wall time: 1min 17s


In [13]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|█████████████████████████████████████████████████████████████████████████| 4177/4177 [00:00<00:00, 1375489.35it/s]


In [14]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|█████████████████████████████████████████████████████████████████████████| 4177/4177 [00:00<00:00, 1043641.38it/s]


Wall time: 1min 55s


In [15]:
model_ug_cbow.save('w2v_model_ug_cbow.word2vec')
model_ug_sg.save('w2v_model_ug_sg.word2vec')

In [16]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')

In [17]:
len(model_ug_cbow.wv.vocab.keys())

33422

In [18]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 33422 word vectors.


In [20]:
!pip install keras

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\manzh\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "C:\Users\manzh\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\manzh\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\manzh\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\manzh\anaconda3\lib\site-packages\pip\_inter

In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [58]:
len(tokenizer.word_index)

42225

In [59]:
for x in x_train[:1]:
    print (x)

the stock market get hit with a double whammy on friday come from a glut of new public offering and a threat from the white house to limit u s investment in china cnbc s jim cramer say the mad money host say busted deal such as peloton smiledirectclub lyft and uber be happen too often and have contribute to a number of down day on wall street in the midst of an ongoing trade war between the world s large economy the public also learn during the day that trump administration official be look at way to deter u s company from invest in chinese company a move that cramer support turn out -pron- s not as sweeping or as negative as the market seem to believe -pron- say the administration doesn t want chinese listing that lack the same kind of transparency as american company and -pron- would prefer investor not to buy the share of company with opaque financial the dow jones industrial average lose more than 70 point or 0 26 while the s p 500 pull back 0 53 and the nasdaq composite drop 1 13 

In [60]:
sequences[:1]

[[1,
  34,
  19,
  195,
  1294,
  16,
  8,
  995,
  10,
  1184,
  234,
  21,
  8,
  3,
  52,
  402,
  475,
  2,
  8,
  2071,
  21,
  1,
  2342,
  915,
  5,
  808,
  123,
  12,
  95,
  7,
  346,
  1074,
  12,
  2320,
  2120,
  60,
  1,
  656,
  1140,
  60,
  445,
  118,
  15,
  1271,
  2,
  2546,
  6,
  1339,
  980,
  1111,
  2,
  13,
  991,
  5,
  8,
  242,
  3,
  216,
  172,
  10,
  791,
  814,
  7,
  1,
  3,
  37,
  1207,
  255,
  1602,
  391,
  1,
  272,
  12,
  212,
  1025,
  1,
  402,
  74,
  896,
  174,
  1,
  172,
  11,
  1212,
  1112,
  1802,
  6,
  45,
  23,
  519,
  5,
  123,
  12,
  18,
  21,
  545,
  7,
  1075,
  18,
  8,
  358,
  11,
  2120,
  373,
  600,
  203,
  4,
  12,
  40,
  15,
  22,
  15,
  687,
  15,
  1,
  19,
  1414,
  5,
  249,
  4,
  60,
  1,
  1112,
  1491,
  148,
  396,
  1075,
  2451,
  11,
  1623,
  1,
  312,
  681,
  3,
  15,
  695,
  18,
  2,
  4,
  144,
  78,
  40,
  5,
  179,
  1,
  31,
  3,
  18,
  16,
  66,
  1,
  1371,
  2575,
  332,
  152,
  892,
 

In [61]:
length = []
for x in x_train:
    length.append(len(x.split()))

In [62]:
max(length)

14743

In [63]:
x_train_seq = pad_sequences(sequences, maxlen=15000)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (3045, 15000)


In [64]:
x_train_seq[:1]

array([[   0,    0,    0, ...,   12, 1480,  181]])

In [65]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=15000)

In [66]:
num_words = 3000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#**I am defining the number of words to be 3000. This means I will only care about 3000 most frequent words in the training set, which has 3045 datapoints. If I don’t limit the number of words, the total number of vocabulary will be more than 40000.**#

In [67]:
np.array_equal(embedding_matrix[34] ,embeddings_index.get('stock'))

True

In [13]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.
C:\Users\manzh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\manzh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\manzh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\manzh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [14]:
model_ptw2v = Sequential()
e = Embedding(3000, 200, weights=[embedding_matrix], input_length=15000, trainable=False)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(1, activation='sigmoid'))
model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=16, verbose=2)

NameError: name 'embedding_matrix' is not defined

In [0]:
model_ptw2v = Sequential()
e = Embedding(3000, 200, input_length=15000)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(1, activation='sigmoid'))
model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Train on 3045 samples, validate on 762 samples
Epoch 1/5


In [0]:
model_ptw2v = Sequential()
e = Embedding(3000, 200, weights=[embedding_matrix], input_length=15000, trainable=True)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(1, activation='sigmoid'))
model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

In [0]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [0]:
structure_test = Sequential()
e = Embedding(3000, 200, input_length=15000)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.summary()

In [0]:
structure_test = Sequential()
e = Embedding(3000, 200, input_length=15000)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

In [0]:
model_cnn_01 = Sequential()
e = Embedding(3000, 200, weights=[embedding_matrix], input_length=15000, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(1, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

In [0]:
model_cnn_02 = Sequential()
e = Embedding(3000, 200, input_length=15000)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_02.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

In [0]:
model_cnn_03 = Sequential()
e = Embedding(3000, 200, weights=[embedding_matrix], input_length=15000, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

In [0]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

tweet_input = Input(shape=(15000,), dtype='int32')

tweet_encoder = Embedding(3000, 200, weights=[embedding_matrix], input_length=15000, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [0]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=32, epochs=5,
                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])

In [0]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.02-0.8333.hdf5')
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

In [0]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=45)

In [0]:
loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

In [0]:
yhat_cnn = loaded_CNN_model.predict(x_test_seq)

In [0]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(y_test, yhat_lr[:,1])
roc_auc = auc(fpr, tpr)
fpr_cnn, tpr_cnn, threshold = roc_curve(y_test, yhat_cnn)
roc_auc_nn = auc(fpr_cnn, tpr_cnn)
plt.figure(figsize=(8,7))
plt.plot(fpr, tpr, label='tfidf-logit (area = %0.3f)' % roc_auc, linewidth=2)
plt.plot(fpr_cnn, tpr_cnn, label='w2v-CNN (area = %0.3f)' % roc_auc_nn, linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic: is positive', fontsize=18)
plt.legend(loc="lower right")
plt.show()